In [1]:
"""
Purpose: 
Will populate the SynapseCompartmentLabels table

"""

'\nPurpose: \nWill populate the SynapseCompartmentLabels table\n\n'

In [2]:
import datajoint as dj
import numpy as np

import datajoint as dj
import numpy as np
from pykdtree.kdtree import KDTree
import time
from tqdm import tqdm


In [3]:
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")
pinky = dj.create_virtual_module("pinky","microns_pinky")

Connecting celiib@10.28.0.34:3306


In [4]:
#combine all the meshes into one mesh
def add_mesh_piece(main_mesh_vertices,main_mesh_faces,sub_mesh_vertices,sub_mesh_faces):
    """
    Purpose: Takes in a large mesh piece and an array of other meshes and 
    returns a large mesh with all meshes appended
    
    Parameters:
    main_mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates
    main_mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices
    sub_mesh_vertices(list of np.arrays) : list of np arrays with the vertices arrays for all subsegments to be added
    sub_mesh_faces(list of np.arrays) : list of np arrays with the faces arrays for all subsegments to be added
    
    Returns:
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for NEW CONCATENATED MESH
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for NEW CONCATENATED MESH
    
    
    Pseudocode: 
    - Checks: 
    a. Make sure there sub_mesh arrays are greater than 0 and of the same length

    1) Count the number of vertices and faces in the main mesh
    2) Iterate through the submesh vertices and faces. In loop:
    a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
    b. Add the vertices_count and add that to every number in the faces array
    c. Concatenate the submesh faces onto the larger mesh face
    d. Save this new vertices and faces as the main_mesh verts and faces
    e. Print out how many new vertices and faces added
    3) Print out number of segments added, total faces/vertices for new mesh
    4) Return the main mesh vertices and faces
    
    """
    #a. Make sure there sub_mesh arrays are greater than 0 and of the same length
    if len(sub_mesh_vertices) <= 0:
        print("There were no vertices in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) <= 0:
        print("There were no face in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) != len(sub_mesh_vertices):
        raise Exception("The sub_mesh_faces and sub_mesh_vertices length did not match")
        
    
    #1) Count the number of vertices and faces in the main mesh
    n_main_vertices = len(main_mesh_vertices)
    n_main_faces = len(main_mesh_faces)
    
    
    #2) Iterate through the submesh vertices and faces. In loop:
    for i,(sub_verts, sub_faces) in enumerate(zip(sub_mesh_vertices,sub_mesh_faces)):
        #a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
        n_sub_verts = len(sub_verts)
        n_sub_faces = len(sub_faces)
        
        main_mesh_vertices = np.vstack([main_mesh_vertices,sub_verts])

        
        #b. Add the vertices_count of main to every number in the faces array
        sub_faces = sub_faces + n_main_vertices
        
        #c. Concatenate the submesh faces onto the larger mesh face
        main_mesh_faces = np.vstack([main_mesh_faces,sub_faces])
        
        #d. Save this new vertices and faces as the main_mesh verts and faces (DONE)
        
        #e. Print out how many new vertices and faces added
        #print(f"Added subsegment {i} with {n_sub_verts} vertices and {n_sub_faces} faces")
        
        n_main_vertices = len(main_mesh_vertices)
        n_main_faces = len(main_mesh_faces)
    
    #3) Print out number of segments added, total faces/vertices for new mesh  
    print(f"Added {len(sub_mesh_vertices)} subsegements \n  --> final mesh: {len(main_mesh_vertices)} vertices and {len(main_mesh_faces)} faces")
        
    return main_mesh_vertices,main_mesh_faces 

In [5]:
def write_synapse_labels_to_database(complete_proofread_key,criteria_id=1):
    #get all the neurons for that user that have complete status (aka have been proofread)
    proofread_segment_ids = (m65.ProofreadLabel & complete_proofread_key).fetch("segment_id")
    proofread_segment_ids


    meshes = dict()
    labels = dict()
    kdtrees = dict()
    mesh_synapses = dict()

    version = complete_proofread_key["version"]
    decimation_ratio = complete_proofread_key["decimation_ratio"]
    author = complete_proofread_key["author"]
    criteria_id=criteria_id 


    for segment_id in proofread_segment_ids:
        # Practicing how to get the mesh and the full subsegments list

        version = complete_proofread_key["version"]
        decimation_ratio = complete_proofread_key["decimation_ratio"]
        author = complete_proofread_key["author"]

        key = dict(segment_id=segment_id, version = version)
        n = (m65.FromNeuromancer & key).fetch1()

        #get all of the segments and their data=
        subsegments = (m65.FromNeuromancer.Subsegment & key).fetch(as_dict=True)
        subsegment_dicts = dict([(k["subsegment_id"],dict(vertices=k["vertices"],faces=k["faces"])) for k in subsegments])

        subsegment_ordered_list = np.sort(np.array(list(subsegment_dicts.keys())))
        subsegments_vertices = [subsegment_dicts[k]["vertices"] for k in subsegment_ordered_list]
        subsegments_faces = [subsegment_dicts[k]["faces"] for k in subsegment_ordered_list]

        # creating the entire mesh from the main mesh and all of its sub meshes: 
        new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
                           main_mesh_faces=n["faces"],
                           sub_mesh_vertices = subsegments_vertices,
                           sub_mesh_faces=subsegments_faces)

        #get the segment labels


        #find the synapses for neuron and all subsegments
        lookup_key = dict(segment_id=segment_id,version = version)
        subsegment_ids = list((m65.FromNeuromancer.Subsegment & lookup_key).fetch("subsegment_id"))
        subsegment_ids.append(segment_id)
        segment_synapses = np.array((m65.Synapse() & [dict(presyn=k) for k in subsegment_ids]).fetch('synapse_id', 'centroid_x', 'centroid_y', 'centroid_z')).T

        #
        labels_search_key = dict(complete_proofread_key,
                                segment_id=segment_id)

        segment_vert_labels, segment_tri_labels = (m65.ProofreadLabel() & labels_search_key).fetch1("vertices","triangles")
        len(segment_vert_labels)

        if len(segment_vert_labels) <= 0:
            print(f"Skipping the mesh {segment_id} because it has no labels")

        #Add the data to the corresponding lists
        meshes[segment_id] = (new_mesh_vertices, new_mesh_faces)
        labels[segment_id] = (segment_vert_labels, segment_tri_labels)
        kdtrees[segment_id] = KDTree(meshes[segment_id][0])
        mesh_synapses[segment_id]  = segment_synapses
        
    matched_synapses = dict()
    synapses_to_exclude = []
    for segment_id in tqdm(proofread_segment_ids):
        synapses = mesh_synapses[segment_id]
        synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
        kdtree = kdtrees[segment_id]
        distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
        vertex_labels = labels[segment_id][0]
        synapse_labels = dict()

        for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
            if distance_node > 5000:
                synapses_to_exclude.append(dict(version=version,synapse_id=synapse_id,
                                               criteria_id=criteria_id))
            else:
                synapse_labels[synapse_id] = vertex_labels[nearest_node]
        matched_synapses[segment_id] = synapse_labels
        
    #add to synapse exclude
    m65.SynapseExclude.insert(synapses_to_exclude,skip_duplicates=True)
    
    all_synapse_labels = list()
    for segment_id,labeled_synapses in tqdm(matched_synapses.items()):
        version_array = np.array([version] * len(labeled_synapses))
        decimation_ratio_array = np.array([decimation_ratio] * len(labeled_synapses))
        segment_id_array = np.array([segment_id] * len(labeled_synapses))
        synapse_to_label = np.array(list(labeled_synapses.items())).T
        proper_label_array = np.array((segment_id_array,
                                       version_array,
                                       decimation_ratio_array,
                                        *synapse_to_label)).T
        all_synapse_labels.extend(proper_label_array)
    all_synapse_labels_array = np.array(all_synapse_labels)

    all_synapse_labels_array = [ [int(seg_id), int(k[1]), float(k[2]), int(k[3]), int(k[4])] for k,seg_id in   zip(all_synapse_labels_array,segment_id_array)]

    start = time.time()
    SynapseCompartmentLabel.insert(all_synapse_labels_array, skip_duplicates=True)
    print(f"Fininshed inserting synapse labels: Took {time.time() - start} seconds")

In [6]:
@schema
class SynapseCompartmentLabel(dj.Manual):
    definition = """
    -> m65.FromNeuromancer
    decimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated
    -> m65.Synapse
    ---
    (presynaptic_label)->m65.LabelKey
    """ # (postsynaptic_label)->pinky.LabelKey : tinyint unsigned

In [7]:
complete_proofread_key = dict(version = 0,
                    decimation_ratio = 1.00,
                    author = "celiib",
                    status="complete")

criteria_id = 1 # ******* RIGHT NOW SET TO A CELII TEST SO NEED TO CHANGE ON REAL RUN******

write_synapse_labels_to_database(complete_proofread_key,criteria_id)

Added 80 subsegements 
  --> final mesh: 1862086 vertices and 3546312 faces
Added 46 subsegements 
  --> final mesh: 1252162 vertices and 2390205 faces


  0%|          | 0/2 [00:00<?, ?it/s]
188it [00:00, 296551.02it/s]

201it [00:00, 226993.84it/s]
100%|██████████| 2/2 [00:00<00:00, 1008.97it/s]

Fininshed inserting synapse labels: Took 0.05410003662109375 seconds
